In [24]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X,K):
    # zip:对最外层的做遍历
    # print(list(zip(X,K)))
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

# X:(2,3,3) K:(2,2,2) Y:(2,2)
X = torch.tensor([[[0.0,1,2],[3,4,5],[6,7,8]],
                  [[1,2,3],[4,5,6],[7,8,9]]])
K = torch.tensor([[[0.0,1],[2.0,3]],[[1,2],[3,4]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [25]:
def corr2d_multi_in_out(X,K):
    '''

    :param X:维度是(2,3,3)
    :param K: 维度是(3,2,2,2)
    :return: 维度是(3,2,2)
    '''
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

# 按照维度0拼接k,k+1,k+2
# K由(2,2,2)变成(3,2,2,2)
K = torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [26]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [29]:
# 1x1卷积
def corr2d_multi_in_out_1x1(X,K):
    '''
    1x1的卷积核
    :param X:(通道数，图片高，图片宽)
    :param K:(卷积核个数，通道数，1，1)
    :return: Y:(卷积核个数，图片高，图片宽)
    '''
    c_i,h,w = X.shape
    # 有几个卷积核就是几维的输出
    c_o = K.shape[0]
    # 把一个通道拉成向量
    X = X.reshape((c_i,h*w))
    # 丢掉1x1部分
    K = K.reshape((c_o,c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
float(torch.abs(Y1-Y2).sum()) < 1e-6

True